In [ ]:
import os, json
from os import listdir, makedirs
from os.path import join, isfile, isdir
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
from data_manipulations import *

### Global variables

In [ ]:
location_list = ['']

In [ ]:
cohort_template = 'Arztbrief/KIS Angaben'
cohort_group = 'Arztbrief/KIS Angaben'
cohort_question = 'Kohorte: Primäre Klasse'
cohort_column = f'{cohort_template}//{cohort_group}::{cohort_question}'

In [ ]:
aggregated_folder = '03_aggregated'
merged_folder = '04_merged'

if not isdir(join('data', aggregated_folder)):
    makedirs(join('data', aggregated_folder))
    
if not isdir(join('data', merged_folder)):
    makedirs(join('data', merged_folder))

### Target variables
* **Target 0:** Höchster Behandlungsstatus (Unique values: unbekannt, ambulant, Notaufnahme, stationär, IMC, ICU)
* **Target 1:** Letzter dokumentierter Patientenoutcome (Unique values: zum Erfassungsdatum noch bestehende Hospitalisation, Entlassung ohne Folgehospitalisation, Entlassung mit Folgehospitalisation, Tod)
* **Target 2:** Höchster Beatmungsbedarf (Unique values: Nasenbrille, nicht-invasive Beatmung, invasive Beatmung, invasive Beatmung mit ECMO)
* **Target 3:** Binärer Schweregrad (Invasive Beatmung und ICU)

In [ ]:
target_col_0 = 'Arztbrief/KIS Angaben//Outcome Parameter::Innerhalb des erfassten Aufenthaltes war der höchste Behandlungsstatus'
target_col_1 = 'Arztbrief/KIS Angaben//Outcome Parameter::Die letzte dokumentierte Patientenoutcomeerfassung beschreibt'
target_col_2 = 'Behandlungsplan//Behandlungsprotokoll::Sauerstofftherapie: Art'

In [ ]:
encoding_0 = {'nicht beantwortet': 0,
              'unbekannt': 0,
              'ambulant': 1,
              'notaufnahme': 2,
              'stationär': 3,
              'imc': 4,
              'icu': 5
             }

encoding_1 = {'nicht beantwortet': 0,
              'unbekannt': 0,
              'zum erfassungsdatum noch bestehende hospitalisation': 1,
              'entlassung ohne folgehospitalisation': 2,
              'entlassung mit folgehospitalisation': 3,
              'tod': 4
             }

encoding_2 = {'nicht beantwortet': 0,
              'unbekannt': 0,
              'nasenbrille': 1,
              'nicht-invasive beatmung': 2,
              'invasive beatmung': 3,
              'invasive beatmung mit ecmo': 4
             }

encoding = [encoding_0, encoding_1, encoding_2]

### Loop through location list

In [ ]:
%%time
for location in location_list:
    file_path = join('data', '02_date_adapted', f'{location}_all_raw_data_risk-model.xlsx')
    
    raw_data = pd.read_excel(file_path)
    data = raw_data.copy()
    
    columns = list(data.columns)

    arztbrief_columns = [col for col in columns if check_template(col, 'Arztbrief/KIS Angaben')]
    behandlungsplan_columns = [col for col in columns if check_template(col, 'Behandlungsplan')]
    ct_columns = [col for col in columns if check_template(col, 'CT')]
    kai_columns = [col for col in columns if check_template(col, 'Klinisch-anamnestische Information')]
    laborparameter_columns = [col for col in columns if check_template(col, 'Laborparameter')]
    roentgen_columns = [col for col in columns if check_template(col, 'Röntgen')]
    
    print(f"Number of unique patients: {data['PatientID'].nunique()}")
    print(f'Number of rows: {len(data)}\n')
    print(f"Value counts of templates: {data.groupby('Template')['PatientID'].count()}\n")
    
    # Filter for COVID patients only
    cohorts = ['Covid-CTs (Fälle am Standort, PCR\\AK pos.)']
    cohort_ids = list(data[data[cohort_column].isin(cohorts)]['PatientID'].unique())
    data = data[data['PatientID'].isin(cohort_ids)]

    print(f"Unique COVID patients: {data['PatientID'].nunique()}")
    
    # Delete "Follow-up" rows
    cohort_remove_value = 'Follow-Up COVID-CTs am Standort'
    data = data[data[cohort_column] != cohort_remove_value]
    
    # Delete "Röntgen" and "Befundzeitpunkt" rows and corresponding columns
    template_remove_values = ['Röntgen', 'Befundzeitpunkt']
    data = data[~data['Template'].isin(template_remove_values)]
    
    data.drop(roentgen_columns, inplace=True, axis=1)

    # Select only patients that have baseline information
    print(data['Klinisch-anamnestische Information//Komorbiditäten aus Arztbrief::Untersuchungstyp'].unique())
    patients = data['PatientID'].unique().tolist()
    baseline_patients = [patient for patient in patients if has_baseline(data, patient)]
    baseline = len(baseline_patients) > 0
    if baseline:
        data = data[data['PatientID'].isin(baseline_patients)]
    print(f'Patients with baseline information: {len(baseline_patients)}')
    non_baseline_patients = list_diff(patients, baseline_patients)
    
    # Select relevant data points for "Behandlungsplan" and "Laborparameter"
    data = data[data['Behandlungsplan//Behandlungsplan::Behandlungsplan: Erhebungsdatum'].isin([np.nan, -1, 0, 1, 2])]
    data = data[data['Laborparameter//Laborparameter::Labordaten: Erhebungsdatum'].isin([np.nan, -1, 0, 1, 2])]
    data.reset_index(inplace=True, drop=True)
    
    # Merge data into one row per patient
    merged_data = pd.DataFrame()
    
    if baseline:
        print('Baseline information available')
        for i, patient in enumerate(baseline_patients):
            tmp_entries = get_entries(data, patient, arztbrief_columns, behandlungsplan_columns, ct_columns, kai_columns, laborparameter_columns, baseline=True)
            merged_data = pd.concat([merged_data, pd.DataFrame(tmp_entries, index=[i])])
    else:
        print('No baseline information available')
        for i, patient in enumerate(patients):
            tmp_entries = get_entries(data, patient, arztbrief_columns, behandlungsplan_columns, ct_columns, kai_columns, laborparameter_columns, baseline=False)
            merged_data = pd.concat([merged_data, pd.DataFrame(tmp_entries, index=[i])])
    
    patient_ids = merged_data.pop('PatientID')
    merged_data.insert(0, 'PatientID', patient_ids)
    
    merged_data.insert(1, 'Location', [location]*len(merged_data))
    
    # Target variables
    merged_data[target_col_0] = merged_data.apply(lambda row: encode(row[target_col_0], encoding[0]), axis=1)
    merged_data[target_col_1] = merged_data.apply(lambda row: encode(row[target_col_1], encoding[1]), axis=1)
    merged_data[target_col_2] = merged_data.apply(lambda row: encode(row[target_col_2], encoding[2]), axis=1)
    
    data[target_col_0] = data.apply(lambda row: encode(row[target_col_0], encoding[0]), axis=1)
    data[target_col_1] = data.apply(lambda row: encode(row[target_col_1], encoding[1]), axis=1)
    data[target_col_2] = data.apply(lambda row: encode(row[target_col_2], encoding[2]), axis=1)
    
    merged_data['Target_0'] = merged_data.apply(lambda row: get_highest_value(data, row['PatientID'], target_col_0), axis=1)
    merged_data['Target_1'] = merged_data.apply(lambda row: get_highest_value(data, row['PatientID'], target_col_1), axis=1)
    merged_data['Target_2'] = merged_data.apply(lambda row: get_highest_value(data, row['PatientID'], target_col_2), axis=1)
    
    merged_data['Target_3'] = merged_data.apply(lambda row: is_severe(row), axis=1)
    
    merged_data.to_excel(join('data', aggregated_folder, f'{location}_all_raw_data_risk-model.xlsx'), index=False)
    
    print(f'{location} saved\n############################################')

### Merge location data into one dataframe

In [ ]:
data_location = join('data', aggregated_folder)
location_files = [f for f in listdir(data_location) if isfile(join(data_location, f))]

In [ ]:
for i, file in enumerate(location_files):
    if i == 0:
        locations_merged = pd.read_excel(join(data_location, file))
    else:
        locations_merged = pd.concat([locations_merged, pd.read_excel(join(data_location, file))])

In [ ]:
# Remove NaN values in target column 2
locations_merged = locations_merged[~locations_merged['Target_2'].isna()]

In [ ]:
t = 0
print(f"NaN values: {locations_merged[f'Target_{t}'].isna().sum()}")
locations_merged[f'Target_{t}'].value_counts()

In [ ]:
t = 2
print(f"NaN values: {locations_merged[f'Target_{t}'].isna().sum()}")
locations_merged[f'Target_{t}'].value_counts()

In [ ]:
t = 3
print(f"NaN values: {locations_merged[f'Target_{t}'].isna().sum()}")
locations_merged[f'Target_{t}'].value_counts()

In [ ]:
locations_merged.to_excel(join('data', merged_folder, f'all_data_combined.xlsx'), index=False)